In [1]:
import os
import gradio as gr
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchRun

# Load .env
load_dotenv()

# Setup models and tools
llm = ChatOpenAI(model="gpt-4o")
search_tool = DuckDuckGoSearchRun()
llm_with_tools = llm.bind_tools([search_tool])

# Define prompt structure
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a knowledgeable and friendly running coach. 
Step 1: Introduce yourself to the user as Motiva, an AI running coach. Tell them that you are here to help them with their running goals and teach them more about running. You should always wait for a response from the user after every question before moving on. Work on understanding their goals. Ask one question at a time and explain that you are asking these questions to personalize the chat. Do not start explaining right away before gathering their information.

Step 2: Look up information that would help you answer their questions and complete their goals. They might offer a running summary of their progress, and to that assess whether their goals are possible. Think step by step and make a plan based on the goal. Once you know about the runner and their goals, advise the runner in an informed, helpful way. Break down the advice and tailor your response and questions to the runner. This will change as the conversation progresses. Once you have enough information, tell the runner you are happy to help them with other things they need help with.

Step 3: Close the conversation. When the runner has no more questions or doesn't need any more help, please insert a motivational quote and tell them you are here to help in the future."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])
chain = prompt | llm_with_tools | StrOutputParser()

# Simulated user profile (persistent between messages)
user_profile = {
    "goal": None,
    "runner_type": None,
    "shoe_mileage": None,
    "injuries": []
}

# Store uploaded running data
running_data_df = None

# Gradio-compatible chat function
def chat_fn(message, history):
    global running_data_df

    # Preprocess input to simulate memory update
    if "goal" in message.lower() and not user_profile["goal"]:
        user_profile["goal"] = message
    if "beginner" in message.lower() or "casual" in message.lower() or "expert" in message.lower():
        user_profile["runner_type"] = message
    if "miles" in message.lower() and any(char.isdigit() for char in message):
        user_profile["shoe_mileage"] = int([int(s) for s in message.split() if s.isdigit()][0])
    if "pain" in message.lower() or "injury" in message.lower():
        user_profile["injuries"].append(message)

    print(f"USER PROFILE: {user_profile}")  # Debug print

    # Inject summary into prompt context if running data is available
    if running_data_df is not None:
        summary = generate_summary(running_data_df)
        message = f"{summary}\n\nUser says: {message}"

    response = chain.invoke({"input": message, "chat_history": history})
    return response

# File upload function
def handle_file(file):
    global running_data_df
    running_data_df = pd.read_csv(file.name)
    print(running_data_df.columns)  # Debug print to inspect column names
    return generate_summary(running_data_df)

# Generate quick summary
def generate_summary(df):
    try:
        df = df.copy()
        total_distance = df['distance'].sum() / 1000  # Convert to km
        avg_distance = df['distance'].mean() / 1000  # Convert to km

        # Filter valid pace entries (distance > 0)
        valid_df = df[(df['distance'] > 0) & (df['moving_time'] > 0)]

        if not valid_df.empty:
            valid_df['pace_min_per_km'] = (valid_df['moving_time'] / valid_df['distance']) * (1000 / 60)
            avg_pace = valid_df['pace_min_per_km'].mean()
        else:
            avg_pace = "N/A"

        total_runs = len(df)

        if isinstance(avg_pace, float) and not pd.isna(avg_pace):
            pace_str = f"{avg_pace:.2f} min/km"
        else:
            pace_str = str(avg_pace)

        summary = f"""\n📊 **Running Data Summary**:
- Total runs: {total_runs}
- Total distance: {total_distance:.2f} km
- Average distance: {avg_distance:.2f} km/run
- Average pace: {pace_str}"""

        return summary
    except Exception as e:
        return f"Error reading file: {e}"

# Launch Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🏃 Running Coach Chatbox")
    file_input = gr.File(label="Upload CSV Running Data", file_types=[".csv"])
    file_output = gr.Textbox(label="Data Summary", lines=6)

    file_input.change(fn=handle_file, inputs=file_input, outputs=file_output)

    gr.ChatInterface(fn=chat_fn, type="messages")

demo.launch(share=True)



* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ee939fc6489dc6384f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
